In [34]:
from pathlib import Path
import torch
torch.set_printoptions(precision=10)

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from models.ranked_transformer import HsqcRankedTransformer

from models.optional_input_ranked_transformer import OptionalInputRankedTransformer
from datasets.optional_2d_folder_dataset import OptionalInputDataModule
import yaml
from pytorch_lightning.loggers import TensorBoardLogger
torch.set_float32_matmul_precision('medium')

Here we compare models' performance over 7 kinds of inputs


In [30]:
"""IMPORTANT !!!
choose model here
"""

# load model 
#  Optional Input Model
is_optional_input = True

# model_path = Path("/root/MorganFP_prediction/reproduce_previous_works/all_2d1d_datasets/both_datasets/R0_R4_reduced_trial1/")
# hyperpaerameters_path = model_path / "hparams.yaml"

# checkpoint_path = model_path / "checkpoints/epoch=30-step=139190.ckpt"

model_path = Path("/root/MorganFP_prediction/reproduce_previous_works/average_3/FP_versions/r0_r4_reduced_trail1")
hyperpaerameters_path = model_path / "hparams.yaml"

checkpoint_path = model_path / "checkpoints/epoch=27-step=47992.ckpt"



### Automation here....
with open(hyperpaerameters_path, 'r') as file:
    hparams = yaml.safe_load(file)
del hparams['checkpoint_path'] # prevent double defition of checkpoint_path
if "use_peak_values" not in hparams:
    hparams['use_peak_values'] = False

model = HsqcRankedTransformer.load_from_checkpoint(checkpoint_path, **hparams)
# model.change_ranker_for_testing()
model.change_ranker_for_testing(test_ranking_set_path = "/workspace/ranking_sets_cleaned_by_inchi/SMILES_R0_to_R4_reduced_FP_ranking_sets_only_all_info_molecules/test/rankingset.pt")



Initialized SignCoordinateEncoder[384] with dims [180, 180, 24] and 2 positional encoders. 24 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args


In [31]:
datamodule = OptionalInputDataModule(dir="/workspace/SMILES_dataset", FP_choice=hparams["FP_choice"], input_src=["HSQC", "oneD_NMR"], batch_size=hparams['bs'], parser_args=hparams)

datamodule.setup("test")
loader_all_inputs, loader_HSQC_H_NMR, loader_HSQC_C_NMR, loader_only_hsqc, loader_only_1d, loader_only_H_NMR, loader_only_C_NMR = \
    datamodule.test_dataloader()
    
model=model.to("cuda")


In [32]:
trainer = Trainer()
metric_to_focus = [
    'rank_1',
    'rank_5',
    "mean_rank",
    'cos',
    'f1'    
]
def show_model_performance(model, out_file ="temp_model_show.txt"):
    loaders = [loader_all_inputs, loader_HSQC_H_NMR, loader_HSQC_C_NMR, loader_only_hsqc, loader_only_1d, loader_only_H_NMR, loader_only_C_NMR]
    names = ["all_inputs", "HSQC_H_NMR", "HSQC_C_NMR", "only_hsqc", "only_1d", "only_H_NMR", "only_C_NMR"]
    with open(out_file, "w") as file:
        for loader, name in zip(loaders, names):
            result =  trainer.test(model, dataloaders=loader)
            file.write(f"\n{name:10}: ")
            for m in metric_to_focus:
                file.write(f"{m}: {result[0]['test/mean_'+m]:.4f}, ")


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [33]:
show_model_performance(model);

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:   2%|▏         | 1/62 [00:00<00:01, 39.09it/s]

Testing DataLoader 0: 100%|██████████| 62/62 [00:01<00:00, 60.17it/s]{'on_epoch': True, 'sync_dist': False} 


Testing DataLoader 0: 100%|██████████| 62/62 [00:01<00:00, 59.32it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    test/mean_accuracy     │     0.989534318447113     │
│   test/mean_active_bits   │    197.58078002929688     │
│     test/mean_ce_loss     │    0.04832196980714798    │
│       test/mean_cos       │    0.8333436846733093     │
│       test/mean_f1        │    0.8425319194793701     │
│    test/mean_mean_rank    │     1.563585638999939     │
│    test/mean_neg_loss     │   0.014580046758055687    │
│    test/mean_pos_loss     │    0.03374192491173744    │
│  test/mean_pos_neg_loss   │    0.04832197353243828    │
│    test/mean_precision    │    0.8853617310523987     │
│     test/mean_rank_1      │    0.9154776930809021     │
│     test/mean_rank_10     │    0.9891632795333862     │
│     test/mean_rank_5      │    0.9818548560142517     │
│     test/mean_recall      │    0.8038829565048218     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Testing DataLoader 0: 100%|██████████| 62/62 [00:00<00:00, 76.56it/s]{'on_epoch': True, 'sync_dist': False} 


Testing DataLoader 0: 100%|██████████| 62/62 [00:00<00:00, 75.98it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    test/mean_accuracy     │    0.9647796750068665     │
│   test/mean_active_bits   │     6.966617584228516     │
│     test/mean_ce_loss     │    0.4547988474369049     │
│       test/mean_cos       │    0.09012208878993988    │
│       test/mean_f1        │    0.03563572093844414    │
│    test/mean_mean_rank    │    1958.9549560546875     │
│    test/mean_neg_loss     │   0.0013279400300234556   │
│    test/mean_pos_loss     │     0.453470915555954     │
│  test/mean_pos_neg_loss   │    0.4547988772392273     │
│    test/mean_precision    │    0.5685029625892639     │
│     test/mean_rank_1      │  0.00025201612152159214   │
│     test/mean_rank_10     │   0.0030241934582591057   │
│     test/mean_rank_5      │   0.0012600806076079607   │
│     test/mean_recall      │   0.018440086394548416    │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 62/62 [00:00<00:00, 72.92it/s]{'on_epoch': True, 'sync_dist': False} 


Testing DataLoader 0: 100%|██████████| 62/62 [00:00<00:00, 72.40it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    test/mean_accuracy     │    0.9698781371116638     │
│   test/mean_active_bits   │     74.5768814086914      │
│     test/mean_ce_loss     │    0.20615233480930328    │
│       test/mean_cos       │    0.4139226973056793     │
│       test/mean_f1        │    0.3655741512775421     │
│    test/mean_mean_rank    │      576.2568359375       │
│    test/mean_neg_loss     │   0.017917508259415627    │
│    test/mean_pos_loss     │    0.1882348209619522     │
│  test/mean_pos_neg_loss   │    0.20615233480930328    │
│    test/mean_precision    │     0.715846836566925     │
│     test/mean_rank_1      │    0.01587701588869095    │
│     test/mean_rank_10     │    0.09613446146249771    │
│     test/mean_rank_5      │    0.05732397735118866    │
│     test/mean_recall      │    0.24638868868350983    │
└───────────────────────────┴───────────────────────────┘

In [35]:
# spectra-flexible model

# load model 

model_path = Path("/root/MorganFP_prediction/reproduce_previous_works/average_3/model_sizes/flexible_384_trail1")
hyperpaerameters_path = model_path / "hparams.yaml"

checkpoint_path = model_path / "checkpoints/epoch=41-step=35994.ckpt"
with open(hyperpaerameters_path, 'r') as file:
    hparams = yaml.safe_load(file)
del hparams['checkpoint_path'] # prevent double defition of checkpoint_path
hparams['use_peak_values'] = False

model = OptionalInputRankedTransformer.load_from_checkpoint(checkpoint_path, **hparams)
model.change_ranker_for_testing()

flexible_model_result = trainer.test(model, datamodule)


Initialized SignCoordinateEncoder[384] with dims [180, 180, 24] and 2 positional encoders. 24 bits are reserved for encoding the final bit
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


HsqcRankedTransformer saving args
Testing DataLoader 6: 100%|██████████| 62/62 [00:00<00:00, 68.27it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃            Test metric            ┃           DataLoader 0            ┃           DataLoader 1            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   test_mean_accuracy_HSQC_C_NMR   │        0.9899526238441467         │        0.9899526238441467         │
│   test_mean_accuracy_HSQC_H_NMR   │        0.9890680909156799         │        0.9890680909156799         │
│   test_mean_accuracy_all_inputs   │        0.9905825257301331         │        0.9905825257301331         │
│    test_mean_accuracy_only_1d     │        0.9897184371948242         │        0.9897184371948242         │
│   test_mean_accuracy_only_C_NMR   │        0.9885189533233643         │        0.9885189533233643         │
│   test_mean_accuracy_only_H_NMR   │        0.9859780073165894         │        0.9859780073165894         │
│   test_mean_accuracy_only_hsqc    │        0.9853689670562744         │        0.9853689670562744         │
│ test_mean_active_bits_HSQC_C_NMR  │         193.2445526123047         │         193.2445526123047         │
│ test_mean_active_bits_HSQC_H_NMR  │         194.1942596435547         │         194.1942596435547         │
│ test_mean_active_bits_all_inputs  │        193.56790161132812         │        193.56790161132812         │
│   test_mean_active_bits_only_1d   │         191.5341033935547         │         191.5341033935547         │
│ test_mean_active_bits_only_C_NMR  │        188.24354553222656         │        188.24354553222656         │
│ test_mean_active_bits_only_H_NMR  │        186.93228149414062         │        186.93228149414062         │
│  test_mean_active_bits_only_hsqc  │        198.64219665527344         │        198.64219665527344         │
│   test_mean_ce_loss_HSQC_C_NMR    │        0.04304837808012962        │        0.04304837808012962        │
│   test_mean_ce_loss_HSQC_H_NMR    │        0.04520231857895851        │        0.04520231857895851        │
│   test_mean_ce_loss_all_inputs    │        0.04041977599263191        │        0.04041977599263191        │
│     test_mean_ce_loss_only_1d     │        0.0441150888800621         │        0.0441150888800621         │
│   test_mean_ce_loss_only_C_NMR    │        0.04957788065075874        │        0.04957788065075874        │
│   test_mean_ce_loss_only_H_NMR    │        0.05705215036869049        │        0.05705215036869049        │
│    test_mean_ce_loss_only_hsqc    │        0.06758617609739304        │        0.06758617609739304        │
│     test_mean_cos_HSQC_C_NMR      │        0.8364850878715515         │        0.8364850878715515         │
│     test_mean_cos_HSQC_H_NMR      │        0.8219335675239563         │        0.8219335675239563         │
│     test_mean_cos_all_inputs      │        0.8470961451530457         │        0.8470961451530457         │
│       test_mean_cos_only_1d       │        0.8318760395050049         │        0.8318760395050049         │
│     test_mean_cos_only_C_NMR      │        0.8101810812950134         │        0.8101810812950134         │
│     test_mean_cos_only_H_NMR      │        0.7686280608177185         │        0.7686280608177185         │
│      test_mean_cos_only_hsqc      │        0.7680350542068481         │        0.7680350542068481         │
│      test_mean_f1_HSQC_C_NMR      │        0.8468981981277466         │        0.8468981981277466         │
│      test_mean_f1_HSQC_H_NMR      │        0.8336454629898071         │        0.8336454629898071         │
│      test_mean_f1_all_inputs      │        0.8565681576728821         │        0.8565681576728821         │
│       test_mean_f1_only_1d        │        0.8426209092140198         │        0.8426209092140198         │
│      test_mean_f1_only_C_NMR      │        0.8229177594184875         │        0.8229177594184875         │
│      test_mean_f1_only_H_NMR      │   

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃            Test metric            ┃           DataLoader 2            ┃           DataLoader 3            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   test_mean_accuracy_HSQC_C_NMR   │        0.9899526238441467         │        0.9899526238441467         │
│   test_mean_accuracy_HSQC_H_NMR   │        0.9890680909156799         │        0.9890680909156799         │
│   test_mean_accuracy_all_inputs   │        0.9905825257301331         │        0.9905825257301331         │
│    test_mean_accuracy_only_1d     │        0.9897184371948242         │        0.9897184371948242         │
│   test_mean_accuracy_only_C_NMR   │        0.9885189533233643         │        0.9885189533233643         │
│   test_mean_accuracy_only_H_NMR   │        0.9859780073165894         │        0.9859780073165894         │
│   test_mean_accuracy_only_hsqc    │        0.9853689670562744         │        0.9853689670562744         │
│ test_mean_active_bits_HSQC_C_NMR  │         193.2445526123047         │         193.2445526123047         │
│ test_mean_active_bits_HSQC_H_NMR  │         194.1942596435547         │         194.1942596435547         │
│ test_mean_active_bits_all_inputs  │        193.56790161132812         │        193.56790161132812         │
│   test_mean_active_bits_only_1d   │         191.5341033935547         │         191.5341033935547         │
│ test_mean_active_bits_only_C_NMR  │        188.24354553222656         │        188.24354553222656         │
│ test_mean_active_bits_only_H_NMR  │        186.93228149414062         │        186.93228149414062         │
│  test_mean_active_bits_only_hsqc  │        198.64219665527344         │        198.64219665527344         │
│   test_mean_ce_loss_HSQC_C_NMR    │        0.04304837808012962        │        0.04304837808012962        │
│   test_mean_ce_loss_HSQC_H_NMR    │        0.04520231857895851        │        0.04520231857895851        │
│   test_mean_ce_loss_all_inputs    │        0.04041977599263191        │        0.04041977599263191        │
│     test_mean_ce_loss_only_1d     │        0.0441150888800621         │        0.0441150888800621         │
│   test_mean_ce_loss_only_C_NMR    │        0.04957788065075874        │        0.04957788065075874        │
│   test_mean_ce_loss_only_H_NMR    │        0.05705215036869049        │        0.05705215036869049        │
│    test_mean_ce_loss_only_hsqc    │        0.06758617609739304        │        0.06758617609739304        │
│     test_mean_cos_HSQC_C_NMR      │        0.8364850878715515         │        0.8364850878715515         │
│     test_mean_cos_HSQC_H_NMR      │        0.8219335675239563         │        0.8219335675239563         │
│     test_mean_cos_all_inputs      │        0.8470961451530457         │        0.8470961451530457         │
│       test_mean_cos_only_1d       │        0.8318760395050049         │        0.8318760395050049         │
│     test_mean_cos_only_C_NMR      │        0.8101810812950134         │        0.8101810812950134         │
│     test_mean_cos_only_H_NMR      │        0.7686280608177185         │        0.7686280608177185         │
│      test_mean_cos_only_hsqc      │        0.7680350542068481         │        0.7680350542068481         │
│      test_mean_f1_HSQC_C_NMR      │        0.8468981981277466         │        0.8468981981277466         │
│      test_mean_f1_HSQC_H_NMR      │        0.8336454629898071         │        0.8336454629898071         │
│      test_mean_f1_all_inputs      │        0.8565681576728821         │        0.8565681576728821         │
│       test_mean_f1_only_1d        │        0.8426209092140198         │        0.8426209092140198         │
│      test_mean_f1_only_C_NMR      │        0.8229177594184875         │        0.8229177594184875         │
│      test_mean_f1_only_H_NMR      │   

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃            Test metric            ┃           DataLoader 4            ┃           DataLoader 5            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   test_mean_accuracy_HSQC_C_NMR   │        0.9899526238441467         │        0.9899526238441467         │
│   test_mean_accuracy_HSQC_H_NMR   │        0.9890680909156799         │        0.9890680909156799         │
│   test_mean_accuracy_all_inputs   │        0.9905825257301331         │        0.9905825257301331         │
│    test_mean_accuracy_only_1d     │        0.9897184371948242         │        0.9897184371948242         │
│   test_mean_accuracy_only_C_NMR   │        0.9885189533233643         │        0.9885189533233643         │
│   test_mean_accuracy_only_H_NMR   │        0.9859780073165894         │        0.9859780073165894         │
│   test_mean_accuracy_only_hsqc    │        0.9853689670562744         │        0.9853689670562744         │
│ test_mean_active_bits_HSQC_C_NMR  │         193.2445526123047         │         193.2445526123047         │
│ test_mean_active_bits_HSQC_H_NMR  │         194.1942596435547         │         194.1942596435547         │
│ test_mean_active_bits_all_inputs  │        193.56790161132812         │        193.56790161132812         │
│   test_mean_active_bits_only_1d   │         191.5341033935547         │         191.5341033935547         │
│ test_mean_active_bits_only_C_NMR  │        188.24354553222656         │        188.24354553222656         │
│ test_mean_active_bits_only_H_NMR  │        186.93228149414062         │        186.93228149414062         │
│  test_mean_active_bits_only_hsqc  │        198.64219665527344         │        198.64219665527344         │
│   test_mean_ce_loss_HSQC_C_NMR    │        0.04304837808012962        │        0.04304837808012962        │
│   test_mean_ce_loss_HSQC_H_NMR    │        0.04520231857895851        │        0.04520231857895851        │
│   test_mean_ce_loss_all_inputs    │        0.04041977599263191        │        0.04041977599263191        │
│     test_mean_ce_loss_only_1d     │        0.0441150888800621         │        0.0441150888800621         │
│   test_mean_ce_loss_only_C_NMR    │        0.04957788065075874        │        0.04957788065075874        │
│   test_mean_ce_loss_only_H_NMR    │        0.05705215036869049        │        0.05705215036869049        │
│    test_mean_ce_loss_only_hsqc    │        0.06758617609739304        │        0.06758617609739304        │
│     test_mean_cos_HSQC_C_NMR      │        0.8364850878715515         │        0.8364850878715515         │
│     test_mean_cos_HSQC_H_NMR      │        0.8219335675239563         │        0.8219335675239563         │
│     test_mean_cos_all_inputs      │        0.8470961451530457         │        0.8470961451530457         │
│       test_mean_cos_only_1d       │        0.8318760395050049         │        0.8318760395050049         │
│     test_mean_cos_only_C_NMR      │        0.8101810812950134         │        0.8101810812950134         │
│     test_mean_cos_only_H_NMR      │        0.7686280608177185         │        0.7686280608177185         │
│      test_mean_cos_only_hsqc      │        0.7680350542068481         │        0.7680350542068481         │
│      test_mean_f1_HSQC_C_NMR      │        0.8468981981277466         │        0.8468981981277466         │
│      test_mean_f1_HSQC_H_NMR      │        0.8336454629898071         │        0.8336454629898071         │
│      test_mean_f1_all_inputs      │        0.8565681576728821         │        0.8565681576728821         │
│       test_mean_f1_only_1d        │        0.8426209092140198         │        0.8426209092140198         │
│      test_mean_f1_only_C_NMR      │        0.8229177594184875         │        0.8229177594184875         │
│      test_mean_f1_only_H_NMR      │   

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃            Test metric            ┃           DataLoader 6            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   test_mean_accuracy_HSQC_C_NMR   │        0.9899526238441467         │
│   test_mean_accuracy_HSQC_H_NMR   │        0.9890680909156799         │
│   test_mean_accuracy_all_inputs   │        0.9905825257301331         │
│    test_mean_accuracy_only_1d     │        0.9897184371948242         │
│   test_mean_accuracy_only_C_NMR   │        0.9885189533233643         │
│   test_mean_accuracy_only_H_NMR   │        0.9859780073165894         │
│   test_mean_accuracy_only_hsqc    │        0.9853689670562744         │
│ test_mean_active_bits_HSQC_C_NMR  │         193.2445526123047         │
│ test_mean_active_bits_HSQC_H_NMR  │         194.1942596435547         │
│ test_mean_active_bits_all_inputs  │        193.56790161132812         │
│   test_mean_active_bits_only_1d   │         191.5341033935547         │
│ test_mean_active_bits_only_C_NMR  │        188.24354553222656         │
│ test_mean_active_bits_only_H_NMR  │        186.93228149414062         │
│  test_mean_active_bits_only_hsqc  │        198.64219665527344         │
│   test_mean_ce_loss_HSQC_C_NMR    │        0.04304837808012962        │
│   test_mean_ce_loss_HSQC_H_NMR    │        0.04520231857895851        │
│   test_mean_ce_loss_all_inputs    │        0.04041977599263191        │
│     test_mean_ce_loss_only_1d     │        0.0441150888800621         │
│   test_mean_ce_loss_only_C_NMR    │        0.04957788065075874        │
│   test_mean_ce_loss_only_H_NMR    │        0.05705215036869049        │
│    test_mean_ce_loss_only_hsqc    │        0.06758617609739304        │
│     test_mean_cos_HSQC_C_NMR      │        0.8364850878715515         │
│     test_mean_cos_HSQC_H_NMR      │        0.8219335675239563         │
│     test_mean_cos_all_inputs      │        0.8470961451530457         │
│       test_mean_cos_only_1d       │        0.8318760395050049         │
│     test_mean_cos_only_C_NMR      │        0.8101810812950134         │
│     test_mean_cos_only_H_NMR      │        0.7686280608177185         │
│      test_mean_cos_only_hsqc      │        0.7680350542068481         │
│      test_mean_f1_HSQC_C_NMR      │        0.8468981981277466         │
│      test_mean_f1_HSQC_H_NMR      │        0.8336454629898071         │
│      test_mean_f1_all_inputs      │        0.8565681576728821         │
│       test_mean_f1_only_1d        │        0.8426209092140198         │
│      test_mean_f1_only_C_NMR      │        0.8229177594184875         │
│      test_mean_f1_only_H_NMR      │        0.7830022573471069         │
│      test_mean_f1_only_hsqc       │        0.7805994153022766         │
│  test_mean_mean_rank_HSQC_C_NMR   │        2.6323859691619873         │
│  test_mean_mean_rank_HSQC_H_NMR   │        6.7318549156188965         │
│  test_mean_mean_rank_all_inputs   │        3.5241353511810303         │
│    test_mean_mean_rank_only_1d    │         4.873836994171143         │
│  test_mean_mean_rank_only_C_NMR   │         8.864221572875977         │
│  test_mean_mean_rank_only_H_NMR   │        26.534603118896484         │
│   test_mean_mean_rank_only_hsqc   │         12.9108247756958          │
│   test_mean_neg_loss_HSQC_C_NMR   │       0.013007685542106628        │
│   test_mean_neg_loss_HSQC_H_NMR   │       0.015253260731697083        │
│   test_mean_neg_loss_all_inputs   │       0.012030345387756824        │
│    test_mean_neg_loss_only_1d     │       0.013238322921097279        │
│   test_mean_neg_loss_only_C_NMR   │       0.014554854482412338        │
│   test_mean_neg_loss_only_H_NMR   │        0.01984979771077633        │
│   test_mean_neg_loss_only_hsqc    │       0.024144992232322693        │
│   test_mean_pos_loss_HSQC_C_NMR   │       0.030040692538022995        │
│   test_mean_pos_loss_HSQC_H_NMR   │       0.029949059709906578        │
│   

In [46]:

names = ["all_inputs", "HSQC_H_NMR", "HSQC_C_NMR", "only_hsqc", "only_1d", "only_H_NMR", "only_C_NMR"]
out_file = "temp_model_show.txt"
with open(out_file, "w") as file:
    for name in  names:
        file.write(f"\n{name:10}: ")
        for m in metric_to_focus:
            file.write(f"{m}: {flexible_model_result[0]['test_mean_'+m+'_'+name]:.4f}, ")


In [44]:
flexible_model_result[0] .keys()    


dict_keys(['test_mean_ce_loss_all_inputs', 'test_mean_pos_loss_all_inputs', 'test_mean_neg_loss_all_inputs', 'test_mean_pos_neg_loss_all_inputs', 'test_mean_cos_all_inputs', 'test_mean_active_bits_all_inputs', 'test_mean_f1_all_inputs', 'test_mean_precision_all_inputs', 'test_mean_recall_all_inputs', 'test_mean_accuracy_all_inputs', 'test_mean_mean_rank_all_inputs', 'test_mean_rank_1_all_inputs', 'test_mean_rank_5_all_inputs', 'test_mean_rank_10_all_inputs', 'test_mean_ce_loss_HSQC_H_NMR', 'test_mean_pos_loss_HSQC_H_NMR', 'test_mean_neg_loss_HSQC_H_NMR', 'test_mean_pos_neg_loss_HSQC_H_NMR', 'test_mean_cos_HSQC_H_NMR', 'test_mean_active_bits_HSQC_H_NMR', 'test_mean_f1_HSQC_H_NMR', 'test_mean_precision_HSQC_H_NMR', 'test_mean_recall_HSQC_H_NMR', 'test_mean_accuracy_HSQC_H_NMR', 'test_mean_mean_rank_HSQC_H_NMR', 'test_mean_rank_1_HSQC_H_NMR', 'test_mean_rank_5_HSQC_H_NMR', 'test_mean_rank_10_HSQC_H_NMR', 'test_mean_ce_loss_HSQC_C_NMR', 'test_mean_pos_loss_HSQC_C_NMR', 'test_mean_neg_loss_